These lines below import the important library used in the process, including several options to overcome huge data problem.

In [ ]:
import lasio
import pandas as pd
import numpy as np
import sys
import os
from pandas import ExcelWriter
np.set_printoptions(threshold=sys.maxsize)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

lasio will automatically try and import pandas

To collect the wireline log start stop and steps. Can be modified to look for another well data information (location, service, etc).

In [ ]:
lines = [line.rstrip('\n') for line in open('WellList.txt')]
location = []
with open('WellStartStopStep.txt', 'w') as f:
    for i in range(len(lines)):
        l = lasio.read(os.path.join("File Location",lines[i]))
        start = l.well.STRT.value
        stop = l.well.STOP.value
        step = l.well.STEP.value
        location.append(lines[i])
        f.write(lines[i]+" "+"%s" % start+" "+"%s" % stop+" "+ "%s" % step+"\n")

If [pandas](http://pandas.pydata.org/) is installed, you can use the df() method to get a pandas [DataFrame](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) version of the data section:

To collect wireline log data availability. The results are categorized into three types, True (if the log is available and complete throughout the wireline interval), False (if the log is available but incomplete throughout the wireline interval), and NaN (if the log is not available). User can filter the log which want to be displayed and also set the threshold in which the log is considered as incomplete.

In [ ]:
lines = [line.rstrip('\n') for line in open('well.txt')]
the_list =pd.DataFrame()
with open('Well_Data.txt', 'w') as f:
    for i in range(len(lines)):
        l = lasio.read(os.path.join("I:","Data Jawa Barat_Eksplorasi","JATIBARANG EOR DATA","wells",lines[i]))
        df = l.df()
        complete =df.isna().sum()<100
        complete.append(pd.Series(lines[i]),ignore_index=True)
        the_list =pd.concat([the_list,complete.to_frame().T],sort=True)
        #f.write(lines[i]+" "+"%s" % the_list+"\n")
        a = the_list.loc[:,the_list.columns.isin(['CAL','CALI','CALS','SP','GR','PEF','LLD','LLS','ILD','SN','MSFL','RXO','RXOZ','SFL','RHOB','RHOZ','DRHO','HDRS','NPHI','TNPH','DT','DTCO','DTL'])]
        f.write("%s" %a)

In [ ]:
nanval = df['GR'].index[df['GR'].apply(np.isnan)]
nanval

**To check the depth of incomplete log, if available

In [ ]:
step = l.well.STEP.value
def remove_adjacent_depth(nums):
    previous = nums[0];
    i = 0
    while i < (len(nums)):
        if nums[i] == previous+step*i:
            nums = np.delete(nums,i)
        else:
            previous = nums[i]
            i=i+1
        print (nums)
    return nums

nanval = remove_adjacent_depth(nanval)
nanval

In [ ]:
print (df['GR'].name,"is not available at",nanval[0],"-",nanval[1],"m")

**To export in excel

In [ ]:
lines = [line.rstrip('\n') for line in open('wellheader_jatibarangformation.txt')]
result=pd.DataFrame([])
with open('Well_HeaderData.txt', 'w') as f:
    for i in range(len(lines)):
        l = lasio.read(os.path.join("I:","Data Jawa Barat_Eksplorasi","LAS",lines[i]))
        param = l.params.dictview()
        dfparam = pd.DataFrame ([param])
        result = pd.concat([result,dfparam])
        print (result)
        writer = ExcelWriter('PythonExport.xlsx')
        dfparam.to_excel(writer,'Sheet1')
        writer.save()

**unfinished